In [1]:
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('../lib'))
from c19us_nyt import df_us as df_nyt
from c19us_jhu import df_us as df_jhu
import numpy as np

In [2]:
df_nyt = df_nyt.reset_index()[['date','fips','day','county','state','cases','deaths']]
print(df_nyt.shape)
df_nyt.head()

(80324, 7)


,date,fips,day,county,state,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [3]:
df_nyt = df_nyt.groupby(['date','fips','day','county','state']).sum().reset_index()
print(df_nyt.shape)
df_nyt.head()

(80324, 7)


,date,fips,day,county,state,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [4]:
df_jhu = df_jhu.reset_index()[['date','fips','day','county','state','cases','deaths','recovered','active']]
print(df_jhu.shape)
df_jhu.head()

(87205, 9)


,date,fips,day,county,state,cases,deaths,recovered,active
0,2020-03-22,36061,0,New York City,New York,9654,63,0,0
1,2020-03-22,36059,0,Nassau,New York,1900,4,0,0
2,2020-03-22,36119,0,Westchester,New York,1873,0,0,0
3,2020-03-22,36103,0,Suffolk,New York,1034,9,0,0
4,2020-03-22,36087,0,Rockland,New York,455,1,0,0


In [5]:
df_jhu = df_jhu.groupby(['date','fips','day','county','state']).sum().reset_index()
print(df_jhu.shape)
df_jhu.head()

(87190, 9)


,date,fips,day,county,state,cases,deaths,recovered,active
0,2020-03-22,1001,0,Autauga,Alabama,0,0,0,0
1,2020-03-22,1003,0,Baldwin,Alabama,2,0,0,0
2,2020-03-22,1005,0,Barbour,Alabama,0,0,0,0
3,2020-03-22,1007,0,Bibb,Alabama,0,0,0,0
4,2020-03-22,1009,0,Blount,Alabama,0,0,0,0


In [6]:
states = pd.read_csv('../../covid-19/data/state_codes.csv')
states.head()

,Code,State
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [7]:
df = pd.merge(df_jhu, df_nyt, on=['date','fips','day','county','state'], how='right')\
    .sort_values('date')
print(df.shape)
df.head()

(80324, 11)


,date,fips,day,county,state,cases_x,deaths_x,recovered,active,cases_y,deaths_y
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,NaN,NaN,1,0


In [8]:
df['cases'] = df['cases_x'].fillna(df['cases_y'])
df['deaths'] = df['deaths_x'].fillna(df['deaths_y'])
df = df.drop(['cases_x','cases_y','deaths_x','deaths_y'], axis=1)
df['recovered'] = df['recovered'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['active'] = df['active'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['cases'] = df['cases'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['deaths'] = df['deaths'].map(lambda x: np.nan if np.isnan(x) else int(x))
print(df.shape)
df.head()

(80324, 9)


,date,fips,day,county,state,recovered,active,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,1,0


In [9]:
df = df.reset_index()[['date','fips','day','county','state','recovered','active','cases','deaths']]
df.columns = ['Date','FIPS','Day','County','State','Recovered','Active','Cases','Deaths']
print(df.shape)
df.head()

(80324, 9)


,Date,FIPS,Day,County,State,Recovered,Active,Cases,Deaths
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,1,0


In [10]:
df_ = pd.merge(df,states,on='State',how='left')

In [11]:
indexNames = df_[df_['County'].isin(['Bronx','Queens','Manhattan','Staten Island','Brooklyn'])].index
df_.drop(indexNames , inplace=True)

In [12]:
all_counties = pd.read_csv('../../covid-19/data/county_fips.csv')
all_counties['County'] = all_counties.County.map(lambda x: x.replace('County','').strip())
all_counties.head()

,FIPS,County,Code
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga,AL
3,1003,Baldwin,AL
4,1005,Barbour,AL


In [13]:
all_counties[all_counties['FIPS']==36061]

,FIPS,County,Code
1892,36061,New York,NY


In [14]:
all_counties.loc[all_counties['FIPS']==36061,'County'] = 'New York City'

In [15]:
for d in sorted(set(df_['Date'])):
    print('Testing... ' + f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv')
    dff = df_[df_['Date'] == d]
    added_counties = pd.merge(dff, all_counties, how='right', on='FIPS')
    added_counties = added_counties.drop(['County_x','Code_x'], axis=1)
    added_counties.columns = ['Date','FIPS','Day','State','Recovered','Active','Cases','Deaths','County','Code']
    added_counties['Cases'] = added_counties['Cases'].map(lambda x: 0 if np.isnan(x) else int(x))
    added_counties['Deaths'] = added_counties['Deaths'].map(lambda x: 0 if np.isnan(x) else int(x))
    added_counties['FIPS'] = added_counties['FIPS'].map(lambda x: '0' + str(x) if (int(x) <= 9999) else str(x))

    added_counties.to_csv(f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv', index=False)

Testing... ../../covid-19/data/county_data/date_2020-01-21.csv
Testing... ../../covid-19/data/county_data/date_2020-01-22.csv
Testing... ../../covid-19/data/county_data/date_2020-01-23.csv
Testing... ../../covid-19/data/county_data/date_2020-01-24.csv
Testing... ../../covid-19/data/county_data/date_2020-01-25.csv
Testing... ../../covid-19/data/county_data/date_2020-01-26.csv
Testing... ../../covid-19/data/county_data/date_2020-01-27.csv
Testing... ../../covid-19/data/county_data/date_2020-01-28.csv
Testing... ../../covid-19/data/county_data/date_2020-01-29.csv
Testing... ../../covid-19/data/county_data/date_2020-01-30.csv
Testing... ../../covid-19/data/county_data/date_2020-01-31.csv
Testing... ../../covid-19/data/county_data/date_2020-02-01.csv
Testing... ../../covid-19/data/county_data/date_2020-02-02.csv
Testing... ../../covid-19/data/county_data/date_2020-02-03.csv
Testing... ../../covid-19/data/county_data/date_2020-02-04.csv
Testing... ../../covid-19/data/county_data/date_2020-02

In [16]:
df2 = df[['Date','Day','State','Recovered','Active','Cases','Deaths']]\
    .groupby(['Date','State','Day'])\
    .sum()\
    .reset_index()

In [17]:
df3 = pd.merge(df2, states, how='left', on='State')
df3['Recovered'] = df3['Recovered'].map(lambda x: int(x))
df3['Active'] = df3['Active'].map(lambda x: int(x))

In [18]:
print(df3.shape)
df3.head()

(2646, 8)


,Date,State,Day,Recovered,Active,Cases,Deaths,Code
0,2020-01-21,Washington,0,0,0,1,0,WA
1,2020-01-22,Washington,1,0,0,1,0,WA
2,2020-01-23,Washington,2,0,0,1,0,WA
3,2020-01-24,Illinois,3,0,0,1,0,IL
4,2020-01-24,Washington,3,0,0,1,0,WA


In [19]:
df3.to_csv('../../covid-19/data/daily_cases_USA_states.csv', index=False)

In [20]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').head()

(2646, 8)


,Date,State,Day,Recovered,Active,Cases,Deaths,Code
0,2020-01-21,Washington,0,0,0,1,0,WA
1,2020-01-22,Washington,1,0,0,1,0,WA
2,2020-01-23,Washington,2,0,0,1,0,WA
3,2020-01-24,Illinois,3,0,0,1,0,IL
4,2020-01-24,Washington,3,0,0,1,0,WA


In [21]:
df4 = df_[['Date', 'County','Recovered','Active','Cases','Deaths','Code']]\
    .groupby(['Date','County','Code'])\
    .sum()\
    .reset_index()
df4['Recovered'] = df4['Recovered'].map(lambda x: int(x))
df4['Active'] = df4['Active'].map(lambda x: int(x))

In [22]:
print(df4.shape)
df4.head()

(80277, 7)


,Date,County,Code,Recovered,Active,Cases,Deaths
0,2020-01-21,Snohomish,WA,0,0,1,0
1,2020-01-22,Snohomish,WA,0,0,1,0
2,2020-01-23,Snohomish,WA,0,0,1,0
3,2020-01-24,Cook,IL,0,0,1,0
4,2020-01-24,Snohomish,WA,0,0,1,0


In [23]:
df4.to_csv('../../covid-19/data/daily_cases_USA_counties.csv', index=False)

In [24]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').head()

(80277, 7)


,Date,County,Code,Recovered,Active,Cases,Deaths
0,2020-01-21,Snohomish,WA,0,0,1,0
1,2020-01-22,Snohomish,WA,0,0,1,0
2,2020-01-23,Snohomish,WA,0,0,1,0
3,2020-01-24,Cook,IL,0,0,1,0
4,2020-01-24,Snohomish,WA,0,0,1,0


In [25]:
df4.Date.min(), df4.Date.max()

(Timestamp('2020-01-21 00:00:00'), Timestamp('2020-04-22 00:00:00'))